# iQuHack 2026 — Entanglement Distillation Game (demo)
This notebook is a **short, hackable** walkthrough from **API key → starting node → distillation circuits → claiming edges**.
You'll mostly edit **one function**: `make_distillation_circuits(...)`.
---
## What the server expects (the important bits)
- You submit **two circuits**: **Alice** and **Bob**. Each acts on **N local qubits** (N = number of raw Bell pairs you request, `1 ≤ N ≤ 8`).
- The server prepares **N noisy Bell pairs** across **2N qubits**, paired “outside-in”:
 - Pair 1: (Alice qubit 0) — (Bob qubit N-1)
 - Pair 2: (Alice qubit 1) — (Bob qubit N-2)
 - …
 - Pair N: (Alice qubit N-1) — (Bob qubit 0) ✅ **this is the output pair**
- After simulating your (Alice ⊗ Bob) circuit, the server computes the **Φ⁺ fidelity** of the final 2-qubit state on:
 - **Alice qubit N-1** and **Bob qubit 0**.
- Measurements are **post-selected to outcome 0** by the server (it projects all measured qubits onto \|0⟩ and renormalizes).
- You can only claim an edge if it connects **owned → unowned** (the game enforces connectivity).
- Budget is only spent on **successful** claims (failed attempts are free).
- To claim an edge, you must satisfy:
 - `fidelity ≥ base_threshold`, **and**
 - `fidelity > baseline + 0.01` (prevents pure identity on some edges).
References for standard purification ideas (great for circuit inspiration):
- BBPSSW (Bennett et al., 1996): https://arxiv.org/abs/quant-ph/9511027
- DEJMPS (Deutsch et al., 1996): https://arxiv.org/abs/quant-ph/9604039
- Review: Dür & Briegel (2007): https://arxiv.org/abs/0705.4165 

## 0) Install / import (run once)
If you're in the `iQuHack2026/` folder, this should work:

In [1]:
# If needed (Jupyter/VS Code):
# %pip install -r requirements.txt

from pathlib import Path
import json
import os

from client import GameClient
from visualization import GraphTool

from qiskit import QuantumCircuit

## 1) Create or load your profile (player_id + API token)
- First run: set `PLAYER_ID`, `NAME`, `LOCATION`, then run the cell to register.
- Later runs: it reloads your saved token from `.iquhack_profile.json`.
⚠️ Don't paste your token into public repos.

In [2]:
PROFILE_PATH = Path(".iquhack_profile.json")

BASE_URL = os.getenv("IQUHACK_BASE_URL", "https://demo-entanglement-distillation-qfhvrahfcq-uc.a.run.app")

# Change these once (first run):
PLAYER_ID = os.getenv("IQUHACK_PLAYER_ID", "team_yourname_here")
NAME = os.getenv("IQUHACK_NAME", "Your Name")
LOCATION = os.getenv("IQUHACK_LOCATION", "remote")  # "remote" or "in_person"

def save_profile(profile: dict):
    PROFILE_PATH.write_text(json.dumps(profile, indent=2))
    print(f"Saved {PROFILE_PATH} (token + player_id).")

def load_profile() -> dict | None:
    if PROFILE_PATH.exists():
        return json.loads(PROFILE_PATH.read_text())
    return None

profile = load_profile()
if profile:
    print("Loaded existing profile:", {k: profile[k] for k in ["base_url", "player_id"]})
    client = GameClient(base_url=profile["base_url"], api_token=profile["api_token"])
    client.player_id = profile["player_id"]
    client.name = profile.get("name")
else:
    client = GameClient(base_url=BASE_URL)
    print("No profile found. We'll register in the next cell.")

No profile found. We'll register in the next cell.


## 2) Register (gets your API token)
If you already loaded a profile above, you can skip this.

In [3]:
if not profile:
    resp = client.register(PLAYER_ID, NAME, location=LOCATION)
    print(resp)

    if resp.get("ok"):
        token = resp["data"]["api_token"]
        save_profile({
            "base_url": BASE_URL,
            "player_id": PLAYER_ID,
            "name": NAME,
            "api_token": token,
        })
    else:
        print("If you get PLAYER_EXISTS, you likely registered before but don't have the token saved.")

{'ok': True, 'data': {'player_id': 'team_yourname_here', 'name': 'Your Name', 'budget': 75, 'location': 'remote', 'starting_candidates': [{'node_id': 'Manila, Philippines', 'utility_qubits': 5, 'bonus_bell_pairs': 1, 'capacity': 7, 'latitude': 14.5995, 'longitude': 120.9842}, {'node_id': 'Leiden, Netherlands', 'utility_qubits': 1, 'bonus_bell_pairs': 3, 'capacity': 5, 'latitude': 52.1601, 'longitude': 4.497}, {'node_id': 'Krakow, Poland', 'utility_qubits': 3, 'bonus_bell_pairs': 2, 'capacity': 6, 'latitude': 50.0647, 'longitude': 19.945}, {'node_id': 'Ahmedabad, India', 'utility_qubits': 4, 'bonus_bell_pairs': 2, 'capacity': 7, 'latitude': 23.0225, 'longitude': 72.5714}], 'api_token': 'Fl8Qt6he-hnKEBT4MFHpiI7sftryv3PJMXEU7pzTPlo'}}
Saved .iquhack_profile.json (token + player_id).


## 3) Choose your starting node (once per run / after restart)
Tip: **Balanced** is often good (utility + bonus).

In [4]:
# Best practice: ask status; if no starting node, you haven't selected yet.
st = client.get_status() if client.player_id else {}
print("Current starting_node:", st.get("starting_node"))

if not st.get("starting_node"):
    # Restart returns your candidate nodes again (handy for reruns)
    r = client.restart()
    candidates = r["data"]["starting_candidates"]
    print("Starting candidates:")
    for c in candidates:
        print(" -", c["node_id"], "| utility:", c["utility_qubits"], "| bonus:", c["bonus_bell_pairs"])

    pick = max(candidates, key=lambda n: n["utility_qubits"] + n["bonus_bell_pairs"])
    print("\nSelecting:", pick["node_id"])
    print(client.select_starting_node(pick["node_id"]))
else:
    print("Already selected a starting node.")

Current starting_node: None
Starting candidates:
 - Manila, Philippines | utility: 5 | bonus: 1
 - Leiden, Netherlands | utility: 1 | bonus: 3
 - Krakow, Poland | utility: 3 | bonus: 2
 - Ahmedabad, India | utility: 4 | bonus: 2

Selecting: Manila, Philippines
{'ok': True, 'data': {'success': True, 'starting_node': 'Manila, Philippines', 'score': 0, 'budget': 75}}


## 4) See your world: status, claimable edges, and a focused graph view

In [5]:
client.print_status(refresh=True)

graph = client.get_cached_graph(force=True)
viz = GraphTool(graph)

owned_nodes = set(client.get_owned_nodes(refresh=True))
viz.print_summary(owned_nodes, focused=True, radius=2)

# Optional: render a focused view (requires matplotlib installed)
viz.render_focused(owned_nodes, radius=2)

PLAYER STATUS: team_yourname_here
Name:           Your Name
Score:          0 points
Budget:         75 bell pairs
Active:         ✅ Yes
Starting node:  Manila, Philippines

Owned nodes:    1
  - Manila, Philippines: 5 qubits, +1 bonus

Owned edges:    0

Claimable edges: 5
  - ['Manila, Philippines', 'Quezon City, Philippines']: threshold=0.93, difficulty=1
  - ['Cebu City, Philippines', 'Manila, Philippines']: threshold=0.88, difficulty=2
  - ['Kaohsiung, Taiwan', 'Manila, Philippines']: threshold=0.88, difficulty=2
  - ['Manila, Philippines', 'Taichung, Taiwan']: threshold=0.82, difficulty=3
  - ['Hong Kong', 'Manila, Philippines']: threshold=0.82, difficulty=3
GRAPH SUMMARY (Focused: radius=2)
Total nodes: 400 (showing: 6)
Total edges: 1057
Owned nodes: 1

NODES:
  [ ] Cebu City, Philippines: 3 qubits, +1 bell pairs
  [ ] Hong Kong: 5 qubits, +3 bell pairs
  [ ] Kaohsiung, Taiwan: 3 qubits, +1 bell pairs
  [✓] Manila, Philippines: 5 qubits, +1 bell pairs
  [ ] Quezon City, Philippi

## 5) Distillation circuits: a solid starting template (BBPSSW / DEJMPS-style)
A common “recurrence” purification move is:
1. Pick a **keep pair** (here: the **output pair** = Alice qubit `N-1`, Bob qubit `0`)
2. Use it as **control** in CNOTs into the other pairs (targets)
3. Measure target halves and **post-select** on 0 outcomes (the server does this for you)
This is inspired by standard entanglement purification protocols (BBPSSW/DEJMPS).
**Important qubit mapping (because of outside-in pairing):**
If Alice target is `t` (0 ≤ t ≤ N-2), then Bob's matching target half is `N-1-t`.

In [6]:
def make_distillation_circuits(num_bell_pairs: int, *, basis: str = "Z") -> tuple[QuantumCircuit, QuantumCircuit]:
    '''
    Returns (circuit_a, circuit_b) for Alice and Bob.

    - num_bell_pairs = N (1..8)
    - basis="Z": parity-check in computational basis (good baseline)
    - basis="X": apply Hadamards first (sometimes helps with phase-flip-ish errors)

    Output pair (the one scored by the server):
      Alice qubit N-1  <->  Bob qubit 0
    '''
    N = int(num_bell_pairs)
    if N < 1 or N > 8:
        raise ValueError("num_bell_pairs must be 1..8")

    # Minimal valid circuits. Classical bits are only used to enable `measure()`.
    qc_a = QuantumCircuit(N, max(N - 1, 1))
    qc_b = QuantumCircuit(N, max(N - 1, 1))

    if basis.upper() == "X":
        for q in range(N):
            qc_a.h(q)
            qc_b.h(q)

    if N == 1:
        # With 1 pair you can't do recurrence distillation; leave (almost) identity.
        return qc_a, qc_b

    control_a = N - 1  # Alice half of output pair
    control_b = 0      # Bob   half of output pair

    for t_a in range(N - 1):
        t_b = (N - 1) - t_a  # Bob's matching half for Alice t_a (outside-in pairing!)
        qc_a.cx(control_a, t_a)
        qc_b.cx(control_b, t_b)

        # Measurements are post-selected to 0 by the server.
        # (Server ignores the classical registers; it only uses "which qubits were measured".)
        qc_a.measure(t_a, t_a)
        qc_b.measure(t_b, t_a)

    return qc_a, qc_b


# Quick sanity-print (useful when debugging mapping)
ca, cb = make_distillation_circuits(2, basis="Z")
print("Alice circuit (2 pairs):")
print(ca)
print("\nBob circuit (2 pairs):")
print(cb)

Alice circuit (2 pairs):
     ┌───┐┌─┐
q_0: ┤ X ├┤M├
     └─┬─┘└╥┘
q_1: ──■───╫─
           ║ 
c: 1/══════╩═
           0 

Bob circuit (2 pairs):
             
q_0: ──■─────
     ┌─┴─┐┌─┐
q_1: ┤ X ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0 


### Easy upgrades to try (edit the function above)

- **Which pair to “keep”**: right now we keep the server-scored pair (Alice `N-1`, Bob `0`). Try making a different pair the “keep” pair, then **SWAP** it into those output positions at the end.
- **More checks**: for `N>2`, your output pair can CNOT into *multiple* targets before measuring them (think “more syndrome bits”).
- **Local basis tricks**: try small local rotations before the CNOTs (`H`, `S`, `X`, `Z`) — DEJMPS-style protocols use local unitaries to reshuffle Bell-basis weight before the recurrence step.
- **Remember**: on this server, measurements are post-selected to 0 and effectively applied after the unitary part, so keep measurements at the end and avoid relying on mid-circuit classical branching.

## 6) Claim an edge (the fun part)
Workflow:
1. Pick a claimable edge.
2. Choose `num_bell_pairs` (start with 2).
3. Build circuits with `make_distillation_circuits(...)`.
4. Submit with `client.claim_edge(...)`.
Tip: failed attempts are free — use that to experiment.

In [7]:
claimable = client.get_claimable_edges(refresh=True)
print(f"{len(claimable)} claimable edges")

owned = set(client.get_owned_nodes(refresh=True))

# Pick a target edge: maximize (unowned node utility + bonus) minus a small penalty for higher threshold
def edge_value(e):
    a, b = e["edge_id"]
    target = b if a in owned else a
    node = client.get_node_info(target) or {}
    return (node.get("utility_qubits", 0) + node.get("bonus_bell_pairs", 0)) - 2.0 * float(e["base_threshold"])

target_edge = max(claimable, key=edge_value)["edge_id"]
edge_info = client.get_edge_info(*target_edge)
print("Target edge:", target_edge, "| threshold:", edge_info["base_threshold"], "| difficulty:", edge_info["difficulty_rating"])

# Heuristic: try X-basis on difficulty==2 sometimes; otherwise Z-basis
basis = "X" if edge_info["difficulty_rating"] == 2 else "Z"
N = 2

circuit_a, circuit_b = make_distillation_circuits(N, basis=basis)

resp = client.claim_edge(tuple(target_edge), circuit_a, circuit_b, num_bell_pairs=N)
print(resp)

client.print_status(refresh=True)

5 claimable edges
Target edge: ['Hong Kong', 'Manila, Philippines'] | threshold: 0.82 | difficulty: 3
{'ok': True, 'data': {'success': True, 'is_valid': False, 'fidelity': 0.9413680781758957, 'threshold': 0.82, 'remaining_budget': 77, 'is_active': True, 'score': 10, 'reward_changes': [{'node_id': 'Hong Kong', 'gained': True, 'utility_delta': 5, 'budget_delta': 3}, {'node_id': 'Manila, Philippines', 'gained': True, 'utility_delta': 5, 'budget_delta': 1}]}}
PLAYER STATUS: team_yourname_here
Name:           Your Name
Score:          10 points
Budget:         77 bell pairs
Active:         ✅ Yes
Starting node:  Manila, Philippines

Owned nodes:    1
  - Manila, Philippines: 5 qubits, +1 bonus

Owned edges:    0

Claimable edges: 5
  - ['Manila, Philippines', 'Quezon City, Philippines']: threshold=0.93, difficulty=1
  - ['Cebu City, Philippines', 'Manila, Philippines']: threshold=0.88, difficulty=2
  - ['Kaohsiung, Taiwan', 'Manila, Philippines']: threshold=0.88, difficulty=2
  - ['Manila, P

## 7) Iterate quickly (sweep a few variants)
This is the *one place* most teams will extend:
- Try `N = 1..8`
- Try `basis = "Z"` vs `"X"`
- Add local gates / extra checks inside `make_distillation_circuits`
The server is deterministic given your circuit, so you can **optimize**.

In [8]:
def try_claim_variants(edge_id, trials=((1, "Z"), (2, "Z"), (2, "X"), (3, "Z"))):
    results = []
    for N, basis in trials:
        ca, cb = make_distillation_circuits(N, basis=basis)
        out = client.claim_edge(tuple(edge_id), ca, cb, num_bell_pairs=N)
        if out.get("ok"):
            d = out["data"]
            results.append({
                "N": N,
                "basis": basis,
                "success": d["success"],
                "fidelity": d["fidelity"],
                "threshold": d["threshold"],
                "remaining_budget": d["remaining_budget"],
                "is_valid": d["is_valid"],
            })
        else:
            results.append({"N": N, "basis": basis, "error": out.get("error")})
    return results

for r in try_claim_variants(target_edge):
    print(r)

{'N': 1, 'basis': 'Z', 'success': True, 'fidelity': 0.8500000000000001, 'threshold': 0.82, 'remaining_budget': 76, 'is_valid': False}
{'N': 2, 'basis': 'Z', 'success': True, 'fidelity': 0.9413680781758957, 'threshold': 0.82, 'remaining_budget': 74, 'is_valid': False}
{'N': 2, 'basis': 'X', 'success': False, 'fidelity': 0.7665173572228445, 'threshold': 0.82, 'remaining_budget': 74, 'is_valid': False}
{'N': 3, 'basis': 'Z', 'success': True, 'fidelity': 0.9784903405696077, 'threshold': 0.82, 'remaining_budget': 71, 'is_valid': False}


## Where to go next (good “fun + education” directions)
- Read the server-side distillation logic:
 - `quantum_function/game/distillation_engine.py` (initial state, post-selection, fidelity extraction)
- Implement a known protocol variant:
 - BBPSSW / recurrence purification
 - DEJMPS (better for Bell-diagonal states)
- Think like a game theorist:
 - Vertex rewards depend on **edge fidelities** (with √rank decay), so a *few very good* edges can beat many mediocre ones.
Have fun — and may your Φ⁺ fidelities be ever in your favor. 🚀⚛️